In [1]:
from meteostream import GefsClient

In [2]:
client = GefsClient(grib_dir="/home/tylerradebaugh/data/grib",
                    idx_list=[7],
                    forecast_hours=(0,12,3))

In [6]:
with client.run_GEFS_pipeline(save_file=None, return_ds=True, clear_gribs=False) as ds:
    print(ds)

Preprocessing files..this might take some time
Processing /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f000.grib2...
Target idx: 7, Name: swh, Tau:000
Creating DataArray for sigCww7
Processing /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f003.grib2...
Target idx: 7, Name: swh, Tau:003
Creating DataArray for sigCww7
Processing /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f006.grib2...
Target idx: 7, Name: swh, Tau:006
Creating DataArray for sigCww7
Processing /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f009.grib2...
Target idx: 7, Name: swh, Tau:009
Creating DataArray for sigCww7
All files processed successfully!
<xarray.Dataset> Size: 33MB
Dimensions:     (valid_time: 4, latitude: 721, longitude: 1440)
Coordinates:
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.3 359.5 359.8
  * valid_time  (valid_time) datetime64[ns]

In [4]:
ds.to_netcdf("test.nc")

In [5]:
file = "/home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f012.grib2"

In [8]:
import numpy as np
import pygrib

output = "testfile.nc"
var_name = "variable_name"  # Replace with your actual variable name


with pygrib.open(file) as grbs:
    grb = grbs.message(5)
    data = grb.values
    lats, lons = grb.latlons()
    valid_time = grb.validDate   # Valid time as datetime object
    ref_time = grb.analDate      # Reference (analysis) time as datetime object

    data = np.expand_dims(data, axis=0)

    print(data.shape)

    # # Create a DataArray with time as an additional dimension
    data_array = xr.DataArray(
        data,  
        coords={
            "latitude": (["latitude", "longitude"], lats),
            "longitude": (["latitude", "longitude"], lons),
            "valid_time": valid_time,
            "ref_time": ref_time
        },
        dims=["valid_time", "latitude", "longitude"],
        name = var_name,
        attrs={
            
            "Description": df.loc[5]['parameter']
            }
    )

    print(data_array)


(1, 721, 1440)
<xarray.DataArray 'variable_name' (valid_time: 1, latitude: 721, longitude: 1440)> Size: 8MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    latitude    (latitude, longitude) float64 8MB 90.0 90.0 90.0 ... -90.0 -90.0
    longitude   (latitude, longitude) float64 8MB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time  datetime64[ns] 8B 2025-02-05T06:00:00
    ref_time    datetime64[ns] 8B 2025-02-04T18:00:00
Attributes:
    Description:  Sig. ht. of combined wind waves and swell


In [5]:
import xarray as xr

with xr.open_dataset("test.nc") as ds:
    print(ds)

<xarray.Dataset> Size: 5GB
Dimensions:     (valid_time: 81, latitude: 721, longitude: 1440)
Coordinates:
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.3 359.5 359.8
  * valid_time  (valid_time) datetime64[ns] 648B 2025-02-04T18:00:00 ... 2025...
    ref_time    datetime64[ns] 8B ...
Data variables:
    sigCww4     (valid_time, latitude, longitude) float64 673MB ...
    sigCww5.5   (valid_time, latitude, longitude) float64 673MB ...
    sigCww7     (valid_time, latitude, longitude) float64 673MB ...
    sigCww9     (valid_time, latitude, longitude) float64 673MB ...
    wsBF8       (valid_time, latitude, longitude) float64 673MB ...
    wsBF9       (valid_time, latitude, longitude) float64 673MB ...
    wsBF10      (valid_time, latitude, longitude) float64 673MB ...
Attributes:
    grb_Param_Index:  [ 5  6  7  8 38 39 40]
    Description:      GEFS Probability Data from NOAA NOMADS HTTPS


In [73]:
ds = xr.open_dataset("testfile.nc")
ds 

<xarray.Dataset> Size: 4MB
Dimensions:        (latitude: 721, longitude: 1440, valid_time: 1, ref_time: 1)
Coordinates:
  * latitude       (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float32 6kB 0.0 0.25 0.5 ... 359.3 359.5 359.8
  * valid_time     (valid_time) datetime64[ns] 8B 2025-02-05
  * ref_time       (ref_time) datetime64[ns] 8B 2025-02-04T12:00:00
Data variables:
    variable_name  (valid_time, latitude, longitude) float32 4MB ...